## <center>**Asignacion 1:** Preprocesamiento de datos</center>

<div style="text-align: justify;">
Antes de comenzar, se realizó una inspección de los datos en la que se encontraron ciertas particularidades en algunas columnas, en primer lugar, la variable 'University Rank' presenta diversos formatos: valores únicos de ranking, rangos cerrados, rangos abiertos e, incluso, la categoría 'Reporter'. Para estandarizar esta información, se decidió dividir la columna en dos: una para la cota inferior y otra para la cota superior. En el caso de los valores representados como '1501+', se asignó como cota superior el número máximo de filas (2341). Por último, para las universidades catalogadas como 'Reporter' (reportadas pero sin ranking), ambos límites se dejarán como valores nulos.</div>

Primero, importaremos los datos.

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv("D:/Universidad/SEMESTRE ACTUAL ii/Mineria/Repo Mineria/DM---UCV---MaximilianoCarrillo/asignacion1/World University Rankings 2023.csv")

Ahora, se va a proceder con el tratado de la columna "University Rank"

In [14]:
def extract_rank_bounds(rank_value):
    if pd.isna(rank_value):
        return pd.Series([np.nan, np.nan])
    
    rank_str = str(rank_value).strip()
    
    # valores unicos
    if rank_str.isdigit():
        rank_num = int(rank_str)
        return pd.Series([rank_num, rank_num])
    
    # rango cerrado con guion
    elif '–' in rank_str:
        parts = rank_str.split('–')
        if len(parts) == 2:
            try:
                lower = int(parts[0].strip())
                upper = int(parts[1].strip())
                return pd.Series([lower, upper])
            except:
                return pd.Series([np.nan, np.nan])
    
    elif '-' in rank_str and not rank_str.endswith('+'): 
        parts = rank_str.split('-')
        if len(parts) == 2:
            try:
                lower = int(parts[0].strip())
                upper = int(parts[1].strip())
                return pd.Series([lower, upper])
            except:
                return pd.Series([np.nan, np.nan])
    
    # rango abierto con +
    elif '+' in rank_str:
        try:
            lower = int(rank_str.replace('+', '').strip())
            upper = 2341
            return pd.Series([lower, upper])
        except:
            return pd.Series([np.nan, np.nan])
    
    # caso reporter
    else:
        return pd.Series([np.nan, np.nan])

# nuevas columnas
data[['rank_lower', 'rank_upper']] = data['University Rank'].apply(extract_rank_bounds)
data = data.drop(columns=['University Rank'])

print(data[['rank_lower', 'rank_upper', 'Name of University']].head())


   rank_lower  rank_upper                     Name of University
0         1.0         1.0                   University of Oxford
1         2.0         2.0                     Harvard University
2         3.0         3.0                University of Cambridge
3         3.0         3.0                    Stanford University
4         5.0         5.0  Massachusetts Institute of Technology


Ahora, se hara algo similar con la columna "Overall Score", ya que tambien tiene valores únicos y rangos.

In [15]:
def extract_score_bounds(score_value):

    if pd.isna(score_value) or str(score_value).strip().lower() in ['n/a', 'nan', '']:
        return pd.Series([np.nan, np.nan])
    
    score_str = str(score_value).strip()
    
    # valor unico
    try:
        score_num = float(score_str)
        return pd.Series([score_num, score_num])
    except ValueError:
        pass
    
    # rango con guion
    if '–' in score_str:
        parts = score_str.split('–')
        if len(parts) == 2:
            try:
                lower = float(parts[0].strip())
                upper = float(parts[1].strip())
                return pd.Series([lower, upper])
            except:
                return pd.Series([np.nan, np.nan])
    
    elif '-' in score_str:
        parts = score_str.split('-')
        if len(parts) == 2:
            try:
                lower = float(parts[0].strip())
                upper = float(parts[1].strip())
                return pd.Series([lower, upper])
            except:
                return pd.Series([np.nan, np.nan])
    
    
    return pd.Series([np.nan, np.nan])

# generar columnas
data[['score_lower', 'score_upper']] = data['OverAll Score'].apply(extract_score_bounds)

data = data.drop(columns=['OverAll Score'])

print(data[['Name of University', 'score_lower', 'score_upper']].head())


                      Name of University  score_lower  score_upper
0                   University of Oxford         96.4         96.4
1                     Harvard University         95.2         95.2
2                University of Cambridge         94.8         94.8
3                    Stanford University         94.8         94.8
4  Massachusetts Institute of Technology         94.2         94.2


Tambien se abordara la comlumna de "Female:Male Ratio", separando el ratio en dos columnas.

In [16]:
def extract_gender_ratio(ratio_value):
    if pd.isna(ratio_value) or str(ratio_value).strip() in ['', 'nan', 'NaN', 'None']:
        return pd.Series([np.nan, np.nan])
    
    ratio_str = str(ratio_value).strip()

    if ':' in ratio_str:
        parts = ratio_str.split(':')
        if len(parts) == 2:
            try:
                female = float(parts[0].strip())
                male = float(parts[1].strip())
                return pd.Series([female, male])
            except:
                pass
    
    return pd.Series([np.nan, np.nan])

data[['female_ratio', 'male_ratio']] = data['Female:Male Ratio'].apply(extract_gender_ratio)


data = data.drop(columns=['Female:Male Ratio'])

print(data[['Name of University', 'female_ratio', 'male_ratio']].head())

                      Name of University  female_ratio  male_ratio
0                   University of Oxford          48.0        52.0
1                     Harvard University          50.0        50.0
2                University of Cambridge          47.0        53.0
3                    Stanford University          46.0        54.0
4  Massachusetts Institute of Technology          40.0        60.0


También se observó que los valores de la variable 'No of student' manejan los decimales de forma incorrecta, lo que derivó en otro error visible en algunos casos: universidades con un número de estudiantes inusualmente bajo (por ejemplo, la Technical University of Munich, con un valor registrado de 33.96). Para corregir esto, se identificaron dichos casos, se procedió a eliminar los decimales y, posteriormente, se multiplicó el valor por mil para obtener la cifra original.

Para corroborar este ajuste, se realizó una pequeña inspección en internet, escogiendo arbitrariamente universidades con números anómalos y verificando su cantidad real de estudiantes y verificando la cercanía con el ajuste. Para este proceso, se definieron como anómalos únicamente los valores menores a 100, ya que la revisión arbitraria de datos mostró que los registros de dos dígitos con decimales eran errores de escala recurrentes, pero para valores entre 100 y 1000, en la mayoria de casos las cifras eran correctas, correspondiendo a instituciones especializadas o para grupos selectos.

Además, para los datos faltantes en esta variable, se obtendrá un valor basado en el promedio de su misma localidad.

In [17]:
def fix_student_scale(value):
    if pd.isna(value) or str(value).strip() in ['', 'nan', 'NaN', 'None']:
        return np.nan
    try:
        num = float(str(value).replace(',', ''))
        
        if 0 < num < 100:
            num = num * 1000
            
        return int(num) 
    except:
        return np.nan

# corrección de escala
data['No of student'] = data['No of student'].apply(fix_student_scale)

valid_data = data[data['No of student'].notna()]
avg_by_location = valid_data.groupby('Location')['No of student'].mean()

def impute_null_students(row):
    current_value = row['No of student']
    location = row['Location']
    
    if pd.isna(current_value) and pd.notna(location):
        if location in avg_by_location:
            return int(avg_by_location[location])
            
    return current_value

data['No of student'] = data.apply(impute_null_students, axis=1)

# verificación
print(data[data['Name of University'].str.contains("Technical University of Munich", na=False)][['Name of University', 'No of student']])



                Name of University  No of student
29  Technical University of Munich        33960.0


Por último, se identificaron diversas observaciones con una ausencia de datos importante, como el nombre de la universidad, el ranking y otros indicadores y debido a que no se cuenta con información como la localidad, no es posible realizar una imputación precisa, por lo tanto cualquier estimación resultaría poco fiable. En estos casos, se optó por eliminar dichas filas, ya que podrian no aportar valor a un análisis

In [18]:
data = data.dropna(subset=['Name of University', 'Location'], how='any')

print(f"registros restantes tras la limpieza: {len(data)}")

registros restantes tras la limpieza: 2047
